In [1]:
from random import randint
import numpy as np


In [2]:
import numpy as np
np.random.seed(500)

In [3]:
def compatibility(i,mi,j,lj):
    if i == j:
        return 0
    if mi == lj:
        if i%9==j%9:
            return 0
        elif int(i/9)==int(j/9):
            return 0
        elif get_block(i)==get_block(j):
            return 0
        else:
            return 1
    else:
        return 1
    
def create_compatibility():
    cm = np.zeros((81,9,81,9))
    for i in range(0,9*9):
        for mi in range(0,9):
            for j in range(0,9*9):
                for lj in range(0,9):
                    cm[i][mi][j][lj] = compatibility(i,mi,j,lj)
    return cm 
            
def get_block(i):
        block_n = 1
        if 0<=int(i/9) and 3>int(i/9):
            block_n *= 3*0
        elif 3<=int(i/9) and 6>int(i/9):
            block_n *= 3*1
        else:
            block_n *= 3*2
            
        if 0<=i%9 and 3>i%9:
            block_n +=0
        elif 3<=i%9 and 6>i%9:
            block_n+=1
        else:
            block_n+=2
            
        return block_n


In [4]:
def update_support_opt():
    global cm,support
    support = (cm*p).sum(axis=(2,3))

def update_prop_opt():
    global support,p
    p = np.multiply(p,support)
    p = p/p.sum(axis=1, keepdims=True)
    
def consistency_opt():
    update_support_opt()
    tot_cons = (support*p).sum()
    return tot_cons

In [5]:
import numpy as np, numpy.random
def init(sk):
    global p,support,cm
    cm = []
    p = np.empty((0,9))
    support = []
    
    cm = create_compatibility()
    
    from scipy.stats import norm
    for i in range(0,9*9):
        row = np.linspace(norm.ppf(1/10), norm.ppf(1/9), 9)
        p = np.append(p, np.array([row/row.sum()]),axis=0)
    
    support_normal = [[0]*9 for i in range(9*9)]
    support = np.asarray(support_normal)
    
    for i in range(0,len(sk)):
        for j in range(0,len(sk)):
            if sk[i][j]!=-1:
                p[i*9+j][sk[i][j]]=1
                for v in range(0,9):
                    if v != sk[i][j]:
                        p[i*9+j][v]=0
                        
                        

def init_matrix(arr):
    sk = [[-1]*9 for i in range(0,9)]
    sk = np.asarray(sk)
    arr = arr.replace(" ","")
    for i in range(0,9):
        for j in range(0,9):
            if arr[i*9+j]!="." and arr[i*9+j]!="0":
                sk[i][j]=int(arr[i*9+j])-1
            else:
                sk[i][j]=-1
    return sk

In [6]:
cm = []
p = np.empty((0,9))
support = []

In [7]:
arr1="... ..3 .17 .15 ..9 ..8 .6. ... ... 1.. ..7 ... ..9 ... 2.. ... 5.. ..4 ... ... .2. 5.. 6.. 34. 34. 2.. ..."
arr2="37. 5.. ..6 ... 36. .12 ... .91 75. ... 154 .7. ..3 .7. 6.. .5. 638 ... .64 98. ... 59. .26 ... 2.. ..5 .64"
arr3="200 080 300 060 070 084 030 500 209 000 105 408 000 000 000 402 706 000 301 007 040 720 040 060 004 010 003"
arr4="000 000 907 000 420 180 000 705 026 100 904 000 050 000 040 000 507 009 920 108 000 034 059 000 507 000 000"
total = [arr1,arr2,arr3,arr4]

In [8]:
def check(sk):
    errors = 0
    for i in range(0,9):
        errors+= sk[i][:].size-np.unique(sk[i][:]).size
        errors+= sk[:][i].size-np.unique(sk[:][i]).size
    for i in range(0,9):
        for j in range(0,9):
            for ki in range(0,9):
                for kj in range(0,9): 
                    if ki!=i and kj!=j:
                        if sk[i][j]==sk[ki][kj] and get_block(i*9+j) == get_block(ki*9+kj):
                            errors+=1
            
    return errors

def return_sol(sk):
    global p
    for i in range(0,9):
        for j in range(0,9):
            #print(np.argmax(p[i*9+j])+1,end=" ")
            sk[i][j] = np.argmax(p[i*9+j])+1
        #print("\n")
    return sk

In [9]:
def solve(arr,my_thre,count):
    current = 0
    peak = 0
    p_count = 100
    sk = init_matrix(arr) 
    init(sk)
    thre = 1
    while thre>my_thre and count>0:
        if p_count == 0:
            print(thre)
            p_count=100
        else:
            p_count-=1
        count-=1
        old_p = p
        consistency_opt()
        update_prop_opt()
        thre = np.linalg.norm(p-old_p)
    return return_sol(sk)

In [10]:
sk=solve(arr2,10e-9,20000)

0.017037643360255844
0.021524390035921562
0.016410929036516735
0.014058569725271352
0.009419965972129473
0.007488522364316199
0.0013968690411959946
0.00011960349565628045
9.346517135326755e-06
7.250588791220662e-07
5.621557250682152e-08


In [11]:
check(sk)

0

In [12]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
for arr in total:
    sk=solve(arr,10e-9,20000)
    print(check(sk))

print(sk)

0.012802467685228464
0.014922385274072188
0.017457011457030137
0.0142920949655478
0.013486393782253348
0.0115108374137679
0.009528941038066011
0.00923155593065674
0.006158649076670425
0.004868010466493859
0.0063376025607621575
0.00580357801763137
0.003193336932568152
0.0012146884858059714
0.00037866760824765354
0.00014582163451398982
0.0003161042115382676
0.0010217864558825444
0.00326766688082784
0.007493966534490313
0.003298587979307716
0.0003388159569496555
2.6912126798465922e-05
2.0904392760341643e-06
1.6209732785784002e-07
1.2570564091058769e-08
4
0.017037643360255844
0.021524390035921562
0.016410929036516735
0.014058569725271352
0.009419965972129473
0.007488522364316199
0.0013968690411959946
0.00011960349565628045
9.346517135326755e-06
7.250588791220662e-07
5.621557250682152e-08
0
0.014614900176441447
0.016824977429494195
0.018232326018943874
0.014249806360071572
0.01136942109058365
0.008063573742143684
0.006853652142740173
0.0042353168237495775
0.00172252324417897
0.0009974864550